In this notebook, we will compute the autocorrelation 

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [2]:
import os
from os import path
import itertools
import pickle

import numpy as np
import pandas as pd
import scipy as sp
from scipy import signal
from tqdm.auto import tqdm
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
from glob import glob
from pandarallel import pandarallel

from IPython.utils.capture import capture_output
with capture_output():
    tqdm.pandas()
    pandarallel.initialize(nb_workers=6)

from ipympl.backend_nbagg import Canvas
Canvas.header_visible.default_value = False

from differentiation import spectral_differentiation as specD

In [3]:
region_sets = {
    'VisCtx' : ['VISp', 'VISl', 'VISrl', 'VISal', 'VISpm', 'VISam'],
    'HVAs' : ['VISl', 'VISrl', 'VISal', 'VISpm', 'VISam'],
    'AllVis' : [
        'LGd', 'LP', 'TH', 'VISp', 'VISl', 'VISrl', 'VISal', 'VISpm', 'VISam'
    ],
    'THx' : ['LGd', 'LP', 'TH'],
    'hipp' : ['CA', 'CA1', 'CA2', 'CA3', 'DG', 'DG-mo', 'DG-po', 'DG-sg'],
}

stim_colors_bg = {
    'spontaneous' : cm.Greys(0.3, 0.3),
    'gabors' : cm.Reds(0.7, 0.3),
    'flashes' : cm.Reds(0.3, 0.3),
    'drifting_gratings' : cm.Blues(0.8, 0.3),
    'drifting_gratings_contrast' : cm.Blues(0.99, 0.3),
    'static_gratings' : cm.Blues(0.5, 0.3),
    'natural_movie_three' : cm.Greens(0.9, 0.3),
    'natural_movie_one' : cm.Greens(0.6, 0.3),
    'natural_movie_one_shuffled' : cm.Purples(0.6, 0.1),
    'Spontaneous' : cm.Greys(0.6, 0.3),
    'Artificial (simple)' : cm.Reds(0.6, 0.3),
    'Artificial (complex)' : cm.Blues(0.8, 0.3),
    'Natural' : cm.Greens(0.8, 0.3),
}

data_directory = (
    '/allen/programs/braintv/workgroups/'
    'tiny-blue-dot/differentiation/refactor/data'
)

session_ids = [
    path.basename(x)
    .strip('.pkl')
    .strip('fr_') for x in glob(
        path.join(data_directory, 'fr_*')
    )
]

In [4]:
def load_fr(session):
    return pd.read_pickle(
        path.join(data_directory, f'fr_{session}.pkl')
    )

def load_units(session):
    return pd.read_pickle(
        path.join(data_directory, f'units_{session}.pkl')
    )

def load_stimulus_table(session):
    return pd.read_pickle(
        path.join(data_directory, f'stimulus_{session}.pkl')
    )

def get_autocorr(arr):
    arr = arr - arr.mean()
    x = np.correlate(arr, arr, mode='full')
    x = x[x.size//2:]/x.max()
    return x

def ln_exp_fit(x, t1, t2, a, c):
    res = np.zeros(len(x))
    res[x<c] = -a - x[x<c]/t1
    res[x>=c] = (x-c)[x>=c]/t2 + (-a-c/t1)
    return res

def exp_fn(x, t, s):
    return s*np.exp(-x/t)

def fit_exp(sig, sr=200):
    x = np.arange(len(sig))/sr
    t, s = sp.optimize.curve_fit(exp_fn, x, sig)[0]
    return t, s

def fit_ac_timescale(ac, sampling_rate=200, thresh=5e-3, lim=40):
    ac = ac[:sampling_rate*2]
    ac2 = ac[1:lim][ac[1:lim]>thresh]
    _x = np.linspace(0, 2, sampling_rate*2, False)[1:lim][ac[1:lim]>thresh]
    res = sp.optimize.curve_fit(
        ln_exp_fit, _x, np.log(ac2),
        bounds=([0, 0.5, -10, 0.03], [1.5, 20, 10, 0.7])
    )
    return res

In [18]:
def apply_ac_time_fitting(
    c, sampling_rate=200, thresh=5e-3, lim=200, ax=None
):
    with capture_output():
        try:
            ac = get_autocorr(c)
            ac = ac[:sampling_rate*2]
            ac2 = ac[1:lim][ac[1:lim]>thresh]
            _x = np.linspace(
                0, 2, sampling_rate*2, False
            )[1:lim][ac[1:lim]>thresh]
            t1, t2, a, c = sp.optimize.curve_fit(
                ln_exp_fit, _x, np.log(ac2),
                bounds=(
                    [0, 0.5, -10, 0.03],
                    [1.5, 20, 10, 0.7]
                )
            )[0]
            
            if ax is not None:
                ax.plot(_x, ac2, label='autocorrelation')
                ax.plot(
                    _x, np.exp(ln_exp_fit(_x, t1, t2, a, c)),
                    label='fit'
                )
                ax.set_xlabel('time (s)', fontsize=9)
                ax.set_ylabel('autocorrelation', fontsize=9)
            
            if (_x<c).sum() < 8:
                return np.nan
            return t1
        except:
            return np.nan

# def apply_ac_time_fitting_stimulus(
#     c, sampling_rate=200, thresh=5e-3, lim=200, ax=None
# ):
#     with capture_output():
#         try:
#             ac = get_autocorr(c)
#             ac = ac[:sampling_rate*2]
#             ac2 = ac[1:lim][ac[1:lim]>thresh]
#             _x = np.linspace(
#                 0, 10, sampling_rate*10, False
#             )[1:lim][ac[1:lim]>thresh]
#             t1, t2, a, c = sp.optimize.curve_fit(
#                 ln_exp_fit, _x, ac2,
#                 bounds=(
#                     [0.1, 0.5, -1, 0.03],
#                     [1.5, 20, 0, 1]
#                 )
#             )[0]
            
#             if ax is not None:
#                 ax.plot(_x, ac2, label='autocorrelation')
#                 ax.plot(
#                     _x, ln_exp_fit(_x, t1, t2, a, c),
#                     label='fit'
#                 )
#                 ax.set_xlabel('time (s)', fontsize=9)
#                 ax.set_ylabel('autocorrelation', fontsize=9)
            
#             if (_x<c).sum() < 8:
#                 return np.nan
#             return -t1*a
#         except:
#             return np.nan

# Apply to stimulus data

In [6]:
stimulus_directory = (
    '/allen/programs/braintv/workgroups/'
    'tiny-blue-dot/differentiation/refactor/stimuli'
)
filenames = {
    'flashes' : 'stim_flash_90',
    'gabors' : 'stim_gabors_90',
    'drifting_gratings' : 'stim_dg_90',
    'drifting_gratings_contrast' : 'stim_dgc_90',
    'static_gratings' : 'stim_sg_90',
    'natural_movie_one' : 'stim_natural_movie_one',
    'natural_movie_one_shuffled' : 'stim_natural_movie_one_shuffled',
    'natural_movie_three' : 'stim_natural_movie_three'
}

## Standard method - get AC(t) and fit to exponential decay

In [49]:
stimulus = 'drifting_gratings_contrast'
f = filenames[stimulus]
movie_data = np.load(path.join(stimulus_directory, f'{f}.npy'), mmap_mode='r')
movie_data = np.reshape(movie_data, (movie_data.shape[0], -1))
movie_data = pd.DataFrame(movie_data, index=np.arange(0, len(movie_data)/200, 1/200))

act = movie_data[
    np.random.choice(movie_data.shape[1], 100, replace=False)
].parallel_apply(
    apply_ac_time_fitting
)

f, ax = plt.subplots(figsize=(3, 2.4), tight_layout=True)
act.plot.hist(ax=ax, bins=50)
act.mean()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.03948890658277803

In [50]:
# apply to all stimuli
acts = {}
for stim, f in tqdm(filenames.items()):
    movie_data = np.load(path.join(stimulus_directory, f'{f}.npy'), mmap_mode='r')
    movie_data = np.reshape(movie_data, (movie_data.shape[0], -1))
    movie_data = pd.DataFrame(movie_data, index=np.arange(0, len(movie_data)/200, 1/200))

    acts[stim] = movie_data[
        np.random.choice(movie_data.shape[1], 100, replace=False)
    ].parallel_apply(
        apply_ac_time_fitting
    ).reset_index(drop=True)
acts = pd.concat(acts, axis=1)

In [68]:
acts['flashes'] = 1
acts['gabors'] = 0.25
acts['static_gratings'] = 0.25
acts['natural_movie_one_shuffled'] = 0.033

In [ ]:
f, ax = plt.subplots(figsize=(5, 1.8), tight_layout=True)
sns.boxplot(
    data=acts[acts.median().sort_values().index],
    linewidth=0.5, showfliers=False, color=cm.Greys(0.4, 0.4)
)
ax.set_xticklabels(
    [x.get_text().replace('_', '\n') for x in ax.get_xticklabels()],
    fontsize=7
)
ax.set_ylabel('characteristic\ntimescale (s)', fontsize=7.8)
# ax.set_yticks([0, 0.5, 1, 1.5])
ax.tick_params(axis='both', labelsize=7);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [70]:
acts.to_pickle(path.join(stimulus_directory, 'stimulus_autocorrelation.pkl'))

## Chunk timeseries into 2 s blocks and compute AC for each block
We will then take the mean $AC(t)$, compute its envelope and use it to obtain a timescale.  
As seen below, this does not reflect the important timescales in the signal.
For example, the signal in the flashes stimulus clearly has a timescale of 0.25s and 1.75s
on which it changes, but the autocorrelation comes out to 0.13s, which is not useful.

In [39]:
stimulus = 'drifting_gratings_contrast'
f = filenames[stimulus]
movie_data = np.load(path.join(stimulus_directory, f'{f}.npy'), mmap_mode='r')
movie_data = np.reshape(movie_data, (movie_data.shape[0], -1))
movie_data = pd.DataFrame(movie_data, index=np.arange(0, len(movie_data)/200, 1/200))

# apply act to multiple segments of the timeseries
ts = movie_data[150].copy()
f, (axts, ax) = plt.subplots(
    2, 1, figsize=(5, 4), tight_layout=True,
    gridspec_kw=dict(height_ratios=[1, 1]), sharex=True
)

# chunk into 4 s blocks
block_size = 2
idx = ts.index.rename('time').to_frame()
idx['block'] = np.array(np.arange(len(ts))/200/block_size, dtype=int)
ts.index = pd.MultiIndex.from_frame(idx)

ts.groupby('block').apply(lambda s: axts.plot(np.arange(len(s))/200, s.values, alpha=0.2))
axts.set_ylabel('signals')
axts.set_title(stimulus)

acs = ts.groupby('block').apply(lambda x: pd.Series(get_autocorr(x))).unstack(1)
acs.columns = acs.columns / 200

acs.T.plot(ax=ax, alpha=0.05, legend=False)
acs.abs().mean().plot(ax=ax, c='k', marker='.', ms=1)
ax.set_ylabel('autocorrelation')
ax.set_xlabel('time (s)');

_t, _s = fit_exp(acs.abs().mean())
x = np.arange(len(acs.mean()))/200
ax.plot(x, exp_fn(x, _t, _s), c='r')
print(_t, _s)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.7715589331084952 0.21544247863431285


In [40]:
apply_ac_time_fitting(acs.mean())

0.01077722736320639

In [12]:
# drifting_gratings_contrast abs().mean().peaks().fit_exp()
print(fit_exp(
    [1, 0.172, 0.125, 0.1, 0.088, 0.077, 0.095, 0.05, 0.04], sr=1
))

# flashes abs().mean().peaks().fit_exp()
print(fit_exp(
#     [1, 0.243, 0.051], sr=1/8
    [1, 0.225, 0.235, 0.137, 0.195], sr=1/2
))

(0.6723914507146633, 0.9932394224498788)
(2.095683466983477, 0.9655789943882744)


## Compute the mean length of contiguous time segments where signal does not change much

In [9]:
# compute all pairwise differences in signals and times
_ts_dists = sp.spatial.distance.pdist(ts.to_frame().values)
_t_dists = sp.spatial.distance.pdist(
    ts.index.get_level_values('time').to_frame().values
)
_t_dists = np.array(np.round(_t_dists*1000), dtype=int)/1000

In [10]:
max_ds = 1
win_size = 3

# discretize the distance matrix
_ts_digitized = np.digitize(_ts_dists, np.arange(0, 255, max_ds))
# convert it into a square form
_ts_digitized = sp.spatial.distance.squareform(_ts_digitized)
# shift the matrix along an axis and test where it is different from original matrix
breaks = np.roll(_ts_digitized, 1, axis=0)!=_ts_digitized
# these are the places where the signal changes
# restrict to win_size time above the diagonal
breaks[np.triu_indices(breaks.shape[0], k=200*win_size)] = False
breaks[np.tril_indices(breaks.shape[0], k=0)] = False
# by counting the breaks we get the number of contiguous segments
n_breaks = breaks.sum(0)
print(
    f'Mean length of contiguous segments is: '
    f'{win_size/np.mean(n_breaks):.2f}+/-{n_breaks.std()*win_size/n_breaks.mean()**2:.2f} s'
)

Mean length of contiguous segments is: 0.03+/-0.01 s


In [11]:
# see the baove in action
# right is the idx array (all time pairs within max_ds are yellow)
# left plot is the breaks array (all edges from the bottom plot are in yellow)
f = plt.figure(figsize=(10, 4), tight_layout=True)
gs = plt.GridSpec(1, 3, width_ratios=[4, 4, 1.5])
ax_sig = f.add_subplot(gs[2])
ax_idx = f.add_subplot(gs[0], sharey=ax_sig)
ax_breaks = f.add_subplot(gs[1], sharex=ax_idx, sharey=ax_idx)

ax_breaks.imshow(
    breaks, aspect='auto', cmap=cm.Greys,
    extent=[0, len(ts)/200, len(ts)/200, 0]
)
ax_breaks.tick_params(axis='y', labelleft=False)
ax_breaks.set_title('break locations')
ax_idx.imshow(
    sp.spatial.distance.squareform(_ts_dists), aspect='auto', cmap=cm.Greys,
    extent=[0, len(ts)/200, len(ts)/200, 0]
)
ax_idx.set_title('pairwise difference in signal')
ax_sig.plot(ts.values, np.arange(len(ts))/200, c='k')
ax_sig.tick_params(axis='y', labelleft=False)
ax_sig.set_title('signal');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [38]:
idx = (np.abs(_ts_dists)<1)&(_t_dists<3)
_tds = pd.Series(_t_dists[_t_dists<3])
norms = _tds.groupby(_tds).size()

f, ax = plt.subplots(figsize=(3, 2.4), tight_layout=True)
res = ax.hist(
    _t_dists[idx], bins=100, density=True,# cumulative=True,
    weights=pd.Series(_t_dists[idx]).map(1/norms)
)
ax.set_xlabel('time difference (s)')
ax.set_ylabel('density');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [41]:
_ts_dists.mean()

23.7690211184572

In [42]:
_medians = []
for i in tqdm(np.logspace(1, 8, 16, base=2)):
    idx = (np.abs(_ts_dists)<i)&(_t_dists<3)
    _medians.append(np.median(_t_dists[idx]))

f, ax = plt.subplots(figsize=(3, 2.4), tight_layout=True)
ax.plot(_medians)
ax.set_xlabel('max change in signal')
ax.set_ylabel('median ')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Directly fitting full AC linearly to get a timescale

In [44]:
f, ax = plt.subplots(1, 1, figsize=(3, 2.4), tight_layout=True)
print(apply_ac_time_fitting(movie_data[20], ax=ax))
# ax.set_yscale('log')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

1.4923238906369978


In [114]:
try:
    acts = pd.read_pickle(
        path.join(stimulus_directory, 'stimulus_autocorrelation.pkl')
    )
except:
    acts = {}
    for stimulus, f in tqdm(filenames.items()):
        movie_data = np.load(path.join(stimulus_directory, f'{f}.npy'))
        movie_data = np.reshape(movie_data, (movie_data.shape[0], -1))
        movie_data = pd.DataFrame(movie_data)

        act = movie_data[
            np.random.choice(movie_data.shape[1], 100, replace=False)
        ].parallel_apply(
            apply_ac_time_fitting_stimulus
        )

        acts[stimulus] = act

    acts = {
        k : v.reset_index(drop=True) for k, v in acts.items()
    }
    acts = pd.concat(acts, axis=1)
    acts.to_pickle(path.join(stimulus_directory, 'stimulus_autocorrelation.pkl'))

In [115]:
f, ax = plt.subplots(figsize=(5, 1.8), tight_layout=True)
sns.boxplot(
    data=acts[acts.mean().sort_values().index],
    linewidth=0.5, showfliers=False, color=cm.Greys(0.4, 0.4)
)
ax.set_xticklabels(
    [x.get_text().replace('_', '\n') for x in ax.get_xticklabels()],
    fontsize=7
)
ax.set_ylabel('autocorrelation\ntimescale (s)', fontsize=7.8)
# ax.set_yticks([0, 0.5, 1, 1.5])
ax.tick_params(axis='both', labelsize=7);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Apply to firing rates

In [72]:
# autocorrelation time for neurons for one example session
session = session_ids[-1]
fr = load_fr(session)
units = load_units(session)
units['idx'] = range(len(units))

stim_table = load_stimulus_table(session)
fr.index = pd.MultiIndex.from_frame(
    stim_table.set_index('time')
    .reindex(fr.index, method='ffill')
    .bfill().rename_axis('time')
    .reset_index()
)

In [7]:
# # test algorithm
# df = fr.xs(4, level='block')
# ac = get_autocorr(df.iloc[:, 1001])
# ac[ac<=0] = 1e-5
# _x = np.linspace(0, 1, 200, False)
# res = fit_ac_timescale(ac, lim=200)

# f, ax = plt.subplots(figsize=(4, 3), tight_layout=True)
# ax.plot(_x[ac[:200]>5e-3], ac[:200][ac[:200]>5e-3], '-o')
# ax.plot(_x[1:], np.exp(ln_exp_fit(_x[1:], *res[0])))
# ax.set_yscale('log', nonpositive='mask')
# ax.set_ylim(0.001, 2)
# print(res[0]*1000)

In [73]:
frm = fr.mean()
frm.index = units.loc[frm.index]\
.set_index('region', append=True).index

top_firing_neurons = frm.groupby('region').apply(
    lambda s: pd.Series(
        s.sort_values()[-20:].index
        .get_level_values(0).values
    )
)
top_firing_neurons

region    
        0     1396
        1      611
        2     1015
        3        4
        4      617
              ... 
VISrl   15    2065
        16    2088
        17    2227
        18    2232
        19    2046
Length: 356, dtype: int64

In [74]:
_neurons = top_firing_neurons.loc[[
    x for x in region_sets['VisCtx'] if x in top_firing_neurons.index.levels[0]
]].unique()

In [75]:
try:
    fr_acs = pd.read_pickle(
        path.join(
            data_directory,
            f'firing_rate_autocorrelation_{session}.pkl'
        )
    )
except:
    fr_acs = fr[_neurons].groupby(
        ['stimulus_name', 'block']
    ).progress_apply(
        lambda df: df.parallel_apply(
            apply_ac_time_fitting
        )
    )
    fr_acs.to_pickle(
        path.join(
            data_directory,
            f'firing_rate_autocorrelation_{session}.pkl'
        )
    )
fr_acs.columns = units.loc[fr_acs.columns].set_index(
    'region', append=True
).index
fr_acs.head(8)

942       945       934       693  \
region                                VISp      VISp      VISp      VISp   
stimulus_name              block                                           
drifting_gratings          2.0    0.115029  0.085385  0.180826  0.097469   
                           5.0    0.110938  0.086486  0.199009       NaN   
                           7.0    0.111007  0.076851  0.195629  0.045796   
drifting_gratings_contrast 15.0   0.116027  0.101281  0.089222  0.141933   
flashes                    1.0    0.102409  0.084869  0.065988  0.084110   
gabors                     0.0    0.144801  0.067747  0.059853  0.014896   
natural_movie_one          4.0    0.109398  0.102675  0.168860  0.008724   
                           12.0   0.099495  0.091025  0.195750  0.119690   

                                       726       918       947       935  \
region                                VISp      VISp      VISp      VISp   
stimulus_name              block                                           
drifting_gratings          2.0    0.111816  0.080467  0.080673  0.083285   
                           5.0    0.117938  0.081733  0.066637  0.090799   
                           7.0    0.122148  0.095215  0.066868  0.149833   
drifting_gratings_contrast 15.0   0.101630  0.064310  0.072330  0.090803   
flashes                    1.0         NaN  0.080182  0.078059  0.098394   
gabors                     0.0    0.077399  0.058854  0.048382  0.062403   
natural_movie_one          4.0    0.167069  0.243174  0.071095  0.129360   
                           12.0   0.124229  0.250779  0.077302       NaN   

                                       943       695  ...       214       67   \
region                                VISp      VISp  ...     VISam     VISam   
stimulus_name              block                      ...                       
drifting_gratings          2.0    0.081402       NaN  ...  0.222549  0.105924   
                           5.0    0.072772       NaN  ...  0.236331  0.016525   
                           7.0    0.094468       NaN  ...  0.317464  0.008276   
drifting_gratings_contrast 15.0   0.077316  0.023438  ...  0.283537       NaN   
flashes                    1.0    0.075988       NaN  ...  0.909640  0.023298   
gabors                     0.0    0.058892       NaN  ...  0.279183  0.083874   
natural_movie_one          4.0    0.072956       NaN  ...  0.215482  0.043621   
                           12.0   0.059432  0.062406  ...  0.268167       NaN   

                                       66        225       226       227  \
region                               VISam     VISam     VISam     VISam   
stimulus_name              block                                           
drifting_gratings          2.0    0.134796  0.074431  0.072105  0.077407   
                           5.0    0.173600  0.092436  0.094173  0.092505   
                           7.0    0.206017  0.104017  0.094237  0.094833   
drifting_gratings_contrast 15.0   0.199067  0.100039  0.097588  0.107004   
flashes                    1.0    0.059975  0.036496  0.058224  0.030765   
gabors                     0.0    0.126953  0.015904  0.013699  0.051030   
natural_movie_one          4.0    0.121894  0.021359  0.053970  0.073583   
                           12.0   0.109864  0.076016  0.089347  0.084261   

                                       202       197       201       60   
region                               VISam     VISam     VISam     VISam  
stimulus_name              block                                          
drifting_gratings          2.0    0.158285  0.125671  0.087300  0.086859  
                           5.0    0.182979  0.140751  0.091691  0.117541  
                           7.0    0.176784  0.131082  0.097408  0.085400  
drifting_gratings_contrast 15.0   0.111187  0.077845  0.083761  0.124839  
flashes                    1.0    0.109364  0.072397  0.074013  0.062879  
gabors                     0.0    0.

In [76]:
fr_acs_mn = fr_acs.groupby('stimulus_name').apply(
    lambda df: np.nanmedian(df.values)
)
fr_acs_sd = fr_acs.groupby('stimulus_name').apply(
    lambda df: np.nanstd(df.values)/np.sqrt(np.size(df.values))
)

In [77]:
# no obvious relationship with mean firing rate
f, ax = plt.subplots(1, 1, figsize=(6, 2.5), tight_layout=True)
ax.plot(fr_acs.groupby('stimulus_name').median().T.values, alpha=0.5)
ax.set_xlabel('unit (ordered by increasing firing rate)', fontsize=10)
ax.set_ylabel('autocorrelation time (s)', fontsize=10);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [78]:
fr_acs_mlt = fr_acs.loc[
    ['spontaneous', 'flashes', 'static_gratings', 'natural_movie_one']
].stack([0, 1]).rename(
    'AC timescale (s)'
).reset_index()
fr_acs_mlt

,stimulus_name,block,level_2,region,AC timescale (s)
0,spontaneous,-1.0,60,VISam,0.194485
1,spontaneous,-1.0,66,VISam,0.263914
2,spontaneous,-1.0,68,VISam,0.088255
3,spontaneous,-1.0,70,VISam,0.120994
4,spontaneous,-1.0,174,VISam,0.033207
...,...,...,...,...,...
641,natural_movie_one,12.0,2218,VISrl,0.054919
642,natural_movie_one,12.0,2221,VISrl,0.101133
643,natural_movie_one,12.0,2226,VISrl,0.085280
644,natural_movie_one,12.0,2227,VISrl,0.111502


In [81]:
f, ax = plt.subplots(figsize=(2.7, 2.7), tight_layout=True)
sns.boxplot(
    hue='stimulus_name', y='AC timescale (s)', x='region',
    data=fr_acs_mlt[fr_acs_mlt.region.isin(['VISp', 'VISam'])],
    showfliers=False, palette=stim_colors_bg, ax=ax
)
ax.set_ylabel('autocorrelation timescale (s)\n(for neuronal firing)', fontsize=8)
ax.set_xlabel('')
# ax.set_xticklabels(['spontaneous', 'static gratings'])
ax.tick_params(labelsize=7)
ax.legend(fontsize=7, frameon=False);

f.savefig('fig_timescales_fr_ac.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Introduction panels

In [158]:
import h5py
data_directory = (
    '/allen/programs/braintv/workgroups/tiny-blue-dot/'
    'differentiation/refactor/behavior/'
)
raw_data_path = path.join(data_directory, 'vis_behavior_npx.hdf5')
mouse_id = '03212019_409096'

spikes = {}
with h5py.File(raw_data_path, 'r') as f:
    spikeTimes = {}
    for probe in f[mouse_id]['ccfRegion'].keys():
        spikeTimes[probe] = {}
        for unit in f[mouse_id]['units'][probe][:]:
            spikeTimes[probe][unit.decode()] = f[mouse_id]['spikeTimes'][probe][unit][:]
    
    for probe in spikeTimes.keys():
        for i, unit in enumerate(spikeTimes[probe].keys()):
            spikes[(probe, unit, i)] = spikeTimes[probe][unit][:, 0]
    spikes = pd.Series(spikes).rename_axis(['probe', 'unit', 'unit_idx'])

In [163]:
hfu = spikes.apply(lambda x: len(x)).sort_values()[-3:].index
spikes.loc[hfu]

probe  unit  unit_idx
C      56    34          [3.7031074603311334, 3.708574087428665, 3.7194...
D      143   81          [3.7027741287054905, 3.7057074405391677, 3.727...
C      70    39          [3.714307379262661, 3.719974004912541, 3.72574...
dtype: object

In [194]:
f, axes = plt.subplots(1, 3, figsize=(3, 1), tight_layout=True, sharey=True)
for i, ax in enumerate(axes):
    for j in range(3):
        r = spikes.iloc[j]
        _fr = pd.Series(np.zeros(500))
        r = r - r[i*500]
        r = r[(r<0.5)&(r>0)]
        _fr[np.array(r*1000, dtype=int)] = 1
        (_fr.rolling(50, win_type='gaussian', center=True).mean(std=8)+j*0.2).plot(ax=ax, c=cm.Greys(0.5, 0.7))
        ax.tick_params(labelleft=False, labelbottom=False, bottom=False, left=False)
        ax.set_xlabel('')
f.savefig('intro_distinct_states.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [234]:
f, axes = plt.subplots(1, 3, figsize=(3, 1), tight_layout=True, sharey=True)
for i, ax in enumerate(axes):
    for j in range(3):
        r = spikes.iloc[j]
        _fr = pd.Series(np.zeros(500))
        r = r - r[0]
        r = r+np.random.rand(len(r))*0.01
        r = r[np.random.choice([False, True], size=len(r), p=[0.2, 0.8])]
        r = r[(r<0.5)&(r>0)]
        _fr[np.array(r*1000, dtype=int)] = 1
        (_fr.rolling(50, win_type='gaussian', center=True).mean(std=8)+j*0.2).plot(ax=ax, c=cm.Greys(0.5, 0.7))
        ax.tick_params(labelleft=False, labelbottom=False, bottom=False, left=False)
        ax.set_xlabel('')
f.savefig('intro_similar_states.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [299]:
f, axes = plt.subplots(1, 3, figsize=(3.5, 3.2), tight_layout=True, sharey=True, gridspec_kw=dict(wspace=0))
for i, ax in enumerate(axes):
    for j in range(3):
        r = spikes.iloc[j]
        _fr = pd.Series(np.zeros(500))
        r = r - r[i*500]
        r = r[(r<0.5)&(r>0)]
        _fr[np.array(r*1000, dtype=int)] = 1
        (_fr.rolling(50, win_type='gaussian', center=True).mean(std=8)+j*0.2).plot(
            ax=ax, c=f'C{i}', alpha=0.7
        )
        ax.tick_params(labelleft=False, labelbottom=False, bottom=False, left=False)
        ax.set_xlabel('')
axes[1].set_xlabel('time\n\nwindow', fontsize=13)
axes[0].set_ylabel('firing rate', fontsize=13)
axes[0].tick_params(axis='y', labelleft=True, pad=-2)
axes[0].set_yticks([0.05, 0.25, 0.45])
axes[0].set_yticklabels(['neuron 0', 'neuron 1', 'neuron 2'], fontsize=9, rotation=60)
axes[0].set_xlabel('state\n', fontsize=13)
axes[0].xaxis.set_label_position('top') 
f.savefig('intro_breakup_states.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [413]:
f, axes = plt.subplots(1, 3, figsize=(3., 2.3), tight_layout=True, sharey=True, gridspec_kw=dict(wspace=0))
for i, ax in enumerate(axes):
    for j in range(3):
        r = spikes.iloc[j]
        _fr = pd.Series(np.zeros(500))
        r = r - r[i*500]
        r = r[(r<0.5)&(r>0)]
        _fr[np.array(r*1000, dtype=int)] = 1
        
        freqs, psd = sp.signal.welch(
            _fr.rolling(50, win_type='gaussian', center=True).mean(std=8).dropna(),
            fs=1000, 
        )
        idx = freqs<100
        psd = psd / psd.max()
        ax.plot(freqs[idx], psd[idx]*0.8+j, c=f'C{i}', alpha=0.7)
        ax.set_xlim(-2, 100)
        
        ax.tick_params(labelleft=False, labelbottom=False, bottom=False, left=False)
        ax.set_xlabel('')
axes[1].set_xlabel('time', fontsize=13)
axes[0].set_ylabel('PSD', fontsize=12)
axes[1].set_title('states in spectral domain')
f.savefig('intro_psd_states.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [410]:
f = plt.figure(figsize=(3, 2.1), tight_layout=True)
gs = plt.GridSpec(1, 5, figure=f, wspace=0, width_ratios=[0.5, 0.5, 0.5, 0.2, 2.8])
axes = [f.add_subplot(gs[0])]
for i in range(1, 3):
    axes.append(f.add_subplot(gs[i], sharey=axes[0]))

psd_arr = [[], [], []]
for i, ax in enumerate(axes):
    for j in range(3):
        r = spikes.iloc[(2-i)]
        _fr = pd.Series(np.zeros(500))
        r = r - r[(2-j)*500]
        r = r[(r<0.5)&(r>0)]
        _fr[np.array(r*1000, dtype=int)] = 1
        
        freqs, psd = sp.signal.welch(
            _fr.rolling(50, win_type='gaussian', center=True).mean(std=8).dropna(),
            fs=1000, 
        )
        psd_arr[i].extend(psd)
        idx = freqs<100
        psd = psd / psd.max()
        ax.plot(freqs[idx], psd[idx]*0.8+j*1.8, c=f'C{(2-j)}', alpha=0.7)
        ax.set_xlim(-2, 100)
        
        ax.axis('off')

ax = f.add_subplot(gs[-1])
pdists = sp.spatial.distance.pdist(psd_arr)

sns.heatmap(
    sp.spatial.distance.squareform(pdists) / np.mean(pdists),
    annot=sp.spatial.distance.squareform(pdists) / np.mean(pdists),
    ax=ax, cbar=False, vmin=0, vmax=0, cmap=cm.Greys, linecolor='k',
    linewidths=0.5
)
ax.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)

axes[1].set_title('state\nvectors', fontsize=11)
ax.set_title('pairwise distance\nbetween states', fontsize=11)
f.savefig('intro_psd_stacked.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [412]:
np.median(pdists)/np.mean(pdists)

0.9869487074084778

In [323]:
_gau = pd.Series([
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
    10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
]).rolling(17, win_type='gaussian', center=True).mean(std=2)

f, ax = plt.subplots(figsize=(1, 1))
ax.axis('off')
_gau.plot(ax=ax, c=cm.Greys(0.6, 0.9), lw=5);
f.savefig('intro_gaussian.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …